In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import os
import numpy as np

print(sklearn.__version__)

0.19.0


In [2]:
# Divide in train and test files [80:20] 

# Directory having content

doc_dir = 'Data/content'

train_file_list = []
test_file_list = []

for f in os.listdir(doc_dir):
    #Random Sampling
    if np.random.uniform(0,1)< 0.8:
        train_file_list.append(f)
    else:
        test_file_list.append(f)

print len(train_file_list)
print len(test_file_list)

1050
257


In [3]:
# Build Tagged dataset to be fed in nltk hmm tagger
# It is a list of tagged sentences. 
# training_data = [ [(word, tag), (word, tag).....]
# [(word, tag), (word, tag).....]
#]

# Directory having tags
tag_dir = 'Data/new_tags'

training_data = []

for f in train_file_list:
    training_sentences =[]
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            pairs_in_line = []
            length = min(len(word_in_file), len(tag_in_file))
            #Create the word_tag pair
            for j in xrange(length):
                pairs_in_line.append((word_in_file[j], tag_in_file[j]));
            training_sentences.append(pairs_in_line)
    if len(training_sentences) > 0:
        training_data.extend(training_sentences)

print training_data[0]

[('Five', 'O'), ('people', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('over', 'O'), ('50', 'O'), ('injured', 'O'), ('in', 'O'), ('three', 'O'), ('blasts', 'O'), ('set', 'O'), ('off', 'O'), ('by', 'O'), ('insurgent', 'O'), ('outfit', 'O'), ('ULFA', 'ORG_Others'), ('here', 'O'), ('today', 'O'), ('hours', 'O'), ('before', 'O'), ('Union', 'O'), ('Home', 'O'), ('Minister', 'O'), ('P', 'PER_Others'), ('Chidambaram', 'PER_Others'), ("'s", 'O'), ('visit', 'O'), ('to', 'O'), ('review', 'O'), ('law', 'O'), ('and', 'O'), ('order', 'O'), ('situation', 'O'), ('in', 'O'), ('the', 'O'), ('state', 'O'), ('rocked', 'O'), ('by', 'O'), ('deadly', 'O'), ('blasts', 'O'), ('that', 'O'), ('left', 'O'), ('88', 'O'), ('dead', 'O'), ('two', 'O'), ('months', 'O'), ('ago', 'O'), ('.', 'O'), ('.', 'O'), ('.', 'O'), ('Three', 'O'), ('people', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('35', 'O'), ('injured', 'O'), (',', 'O'), ('including', 'O'), ('four', 'O'), ('women', 'O'), (',', 'O'), ('when'

In [21]:
def word2features(sent, i):
    word = sent[i][0]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
    ]
    if i > 0:
        word1 = sent[i-1][0]
        tag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
           # '-1:tag1=' + tag1,
        ])
    else:
        features.append('BOS')
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
        ])
    else:
        features.append('EOS')
        
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2tags(sent):
    return [tag for word, tag in sent]

def sent2words(sent):
    return [word for word, tag in sent] 

In [22]:
print training_data[0]
print sent2features(training_data[0])[0]
print sent2tags(training_data[0])[0]
print sent2words(training_data[0])[0]

[('Five', 'O'), ('people', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('over', 'O'), ('50', 'O'), ('injured', 'O'), ('in', 'O'), ('three', 'O'), ('blasts', 'O'), ('set', 'O'), ('off', 'O'), ('by', 'O'), ('insurgent', 'O'), ('outfit', 'O'), ('ULFA', 'ORG_Others'), ('here', 'O'), ('today', 'O'), ('hours', 'O'), ('before', 'O'), ('Union', 'O'), ('Home', 'O'), ('Minister', 'O'), ('P', 'PER_Others'), ('Chidambaram', 'PER_Others'), ("'s", 'O'), ('visit', 'O'), ('to', 'O'), ('review', 'O'), ('law', 'O'), ('and', 'O'), ('order', 'O'), ('situation', 'O'), ('in', 'O'), ('the', 'O'), ('state', 'O'), ('rocked', 'O'), ('by', 'O'), ('deadly', 'O'), ('blasts', 'O'), ('that', 'O'), ('left', 'O'), ('88', 'O'), ('dead', 'O'), ('two', 'O'), ('months', 'O'), ('ago', 'O'), ('.', 'O'), ('.', 'O'), ('.', 'O'), ('Three', 'O'), ('people', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('35', 'O'), ('injured', 'O'), (',', 'O'), ('including', 'O'), ('four', 'O'), ('women', 'O'), (',', 'O'), ('when'

In [23]:
x_train = [sent2features(s) for s in training_data]
y_train = [sent2tags(s) for s in training_data]

In [24]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(x_train, y_train):
    trainer.append(xseq, yseq)

In [25]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [26]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [27]:
# Train and save the model
trainer.train('osint-bomb-blast.crfsuite')

In [28]:
tagger = pycrfsuite.Tagger()
tagger.open('osint-bomb-blast.crfsuite')

In [29]:
# Create test data
test_data = []
for f in test_file_list:
    test_sentences =[]
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            pairs_in_line = []
            length = min(len(word_in_file), len(tag_in_file))
            #Create the word_tag pair
            for j in xrange(length):
                pairs_in_line.append((word_in_file[j], tag_in_file[j]));
            test_sentences.append(pairs_in_line)
    if len(test_sentences) > 0:
        test_data.extend(test_sentences)

print test_data[0]

[('Atiq', 'PER_Others'), ('Khan', 'PER_Others'), ('.', 'O'), ('.', 'O'), ('LUCKNOW', 'LOC_Others'), (':', 'O'), ('The', 'ORG_Others'), ('Anti-Terrorism', 'ORG_Others'), ('Squad', 'ORG_Others'), ('-LRB-', 'O'), ('ATS', 'ORG_Others'), ('-RRB-', 'O'), (',', 'O'), ('Mumbai', 'LOC_Others'), (',', 'O'), ('on', 'O'), ('Thursday', 'O'), ('secured', 'O'), ('the', 'O'), ('transit', 'O'), ('remand', 'O'), ('for', 'O'), ('Sudhakar', 'PER_Accused'), ('Dwivedi', 'PER_Accused'), (',', 'O'), ('alias', 'O'), ('Amritanand', 'PER_Accused'), (',', 'O'), ('a', 'O'), ('suspect', 'O'), ('in', 'O'), ('the', 'O'), ('Malegaon', 'LOC_Event'), ('bomb', 'O'), ('blast', 'O'), ('case', 'O'), ('of', 'O'), ('September', 'O'), (',', 'O'), ('2008', 'O'), ('.', 'O'), ('The', 'O'), ('remand', 'O'), ('till', 'O'), ('November', 'O'), ('16', 'O'), ('was', 'O'), ('granted', 'O'), ('by', 'O'), ('Mukesh', 'PER_Others'), ('Kumar', 'PER_Others'), (',', 'O'), ('Civil', 'O'), ('Judge', 'O'), (',', 'O'), ('Junior', 'O'), ('Division'

In [30]:
example_sent = test_data[0]
print(' '.join(sent2words(example_sent)))

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2tags(example_sent)))

Atiq Khan . . LUCKNOW : The Anti-Terrorism Squad -LRB- ATS -RRB- , Mumbai , on Thursday secured the transit remand for Sudhakar Dwivedi , alias Amritanand , a suspect in the Malegaon bomb blast case of September , 2008 . The remand till November 16 was granted by Mukesh Kumar , Civil Judge , Junior Division , Lucknow . . . His name had earlier been disclosed as Dayanand Pandey but , following calls from the DGPs office late on Wednesday night , it was given as Sudhakar Dwivedi , son of Dayanand Dwivedi . He had taken the name Amritanand and become the Peethadheeshwar of the Sharda Sarvagya Peeth in Jammu . . . Dwivedi was picked up by the Mumbai ATS from the residence of his brother , Pushkar Dwivedi , at Rawatpur village in Kanpur on Wednesday . He is to be taken to Mumbai for further interrogation . . . Dwivedi was brought to Lucknow on Wednesday night and was subjected to a detailed interrogation . Details of the closed-door interrogation were not known , but sources said the ATS wa

In [31]:
def report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [32]:
x_test = [sent2features(s) for s in test_data]
y_test = [sent2tags(s) for s in test_data]

In [33]:
%%time
y_pred = [tagger.tag(xseq) for xseq in x_test]

CPU times: user 972 ms, sys: 0 ns, total: 972 ms
Wall time: 971 ms


In [34]:
print(report(y_test, y_pred))

             precision    recall  f1-score   support

    Accused       0.00      0.00      0.00        46
      Event       0.00      0.00      0.00        93
LOC_Accused       0.00      0.00      0.00       178
  LOC_Assoc       0.00      0.00      0.00        93
  LOC_Event       0.52      0.36      0.42      1338
 LOC_Others       0.52      0.51      0.51      1724
 LOC_Victim       0.00      0.00      0.00        27
   Location       0.00      0.00      0.00       157
ORG_Accused       0.74      0.67      0.70       613
  ORG_Assoc       0.00      0.00      0.00        27
 ORG_Others       0.68      0.52      0.59      2122
 ORG_Victim       0.00      0.00      0.00       150
PER_Accused       0.53      0.42      0.47       812
  PER_Assoc       0.00      0.00      0.00        30
 PER_Others       0.73      0.67      0.70      1689
 PER_Victim       0.47      0.16      0.24       251
     Victim       0.00      0.00      0.00        11

avg / total       0.56      0.47      0.51  

In [35]:
num_test_data = len(y_test)
print len(y_pred)
print num_test_data

257
257


In [36]:
# Test
actual_tag_count = dict()
matched_tag_count = dict()
matched_tag_count['PER_Others'] = 0
actual_tag_count['PER_Others'] = 0
matched_tag_count['PER_Victim'] = 0
actual_tag_count['PER_Victim'] = 0
matched_tag_count['PER_Accused'] = 0
actual_tag_count['PER_Accused'] = 0
matched_tag_count['ORG_Victim'] = 0
actual_tag_count['ORG_Victim'] = 0
matched_tag_count['ORG_Accused'] = 0
actual_tag_count['ORG_Accused'] = 0
matched_tag_count['ORG_Others'] = 0
actual_tag_count['ORG_Others'] = 0
matched_tag_count['LOC_Accused'] = 0
actual_tag_count['LOC_Accused'] = 0
matched_tag_count['LOC_Others'] = 0
actual_tag_count['LOC_Others'] = 0
matched_tag_count['LOC_Event'] = 0
actual_tag_count['LOC_Event'] = 0
matched_tag_count['LOC_Victim'] = 0
actual_tag_count['LOC_Victim'] = 0

for i in xrange(num_test_data):
    y_test_i = y_test[i]
    y_pred_i = y_pred[i]
    test_tag_length = len(y_test[i])
    predicted_tag_length = len(y_pred[i])
    for j in xrange(test_tag_length):
        if y_test_i[j] in actual_tag_count.keys():
            actual_tag_count[y_test_i[j]] = actual_tag_count[y_test_i[j]] + 1
        
        if y_test_i[j] == y_pred_i[j]:
            if y_test_i[j] in matched_tag_count.keys():
                matched_tag_count[y_test_i[j]] = matched_tag_count[y_test_i[j]] + 1

In [37]:
print('======= Accuracy Class Wise =====================\n')

print('Class    Matched Total %')
print('--------------------------')
print ('PER_Others: '+str(matched_tag_count['PER_Others'])+ ' ' + str(actual_tag_count['PER_Others'])+ ' ' + str(matched_tag_count['PER_Others']*100/actual_tag_count['PER_Others'])+'%')
print ('PER_Victim: '+str(matched_tag_count['PER_Victim'])+ ' ' + str(actual_tag_count['PER_Victim'])+ ' '+str(matched_tag_count['PER_Victim']*100/actual_tag_count['PER_Victim'])+'%')
print ('PER_Accused: '+str(matched_tag_count['PER_Accused'])+ ' ' + str(actual_tag_count['PER_Accused'])+ ' '+str(matched_tag_count['PER_Accused']*100/actual_tag_count['PER_Accused'])+'%')
print ('ORG_Victim: '+str(matched_tag_count['ORG_Victim']) + ' '+ str(actual_tag_count['ORG_Victim'])+ ' '+str(matched_tag_count['ORG_Victim']*100/actual_tag_count['ORG_Victim'])+'%')
print ('ORG_Accused: '+str(matched_tag_count['ORG_Accused']) + ' '+ str(actual_tag_count['ORG_Accused'])+ ' '+str(matched_tag_count['ORG_Accused']*100/actual_tag_count['ORG_Accused'])+'%')
print ('ORG_Others: '+str(matched_tag_count['ORG_Others']) + ' '+ str(actual_tag_count['ORG_Others'])+ ' '+str(matched_tag_count['ORG_Others']*100/actual_tag_count['ORG_Others'])+'%')
print ('LOC_Accused: '+str(matched_tag_count['LOC_Accused'])+ ' ' + str(actual_tag_count['LOC_Accused'])+ ' '+str(matched_tag_count['LOC_Accused']*100/actual_tag_count['LOC_Accused'])+'%')
print ('LOC_Others: '+str(matched_tag_count['LOC_Others'])+ ' ' + str(actual_tag_count['LOC_Others'])+ ' '+str(matched_tag_count['LOC_Others']*100/actual_tag_count['LOC_Others'])+'%')
print ('LOC_Event: '+str(matched_tag_count['LOC_Event']) + ' '+ str(actual_tag_count['LOC_Event'])+ ' '+str(matched_tag_count['LOC_Event']*100/actual_tag_count['LOC_Event'])+'%')
print ('LOC_Victim: '+str(matched_tag_count['LOC_Victim']) + ' '+ str(actual_tag_count['LOC_Victim'])+ ' '+str(matched_tag_count['LOC_Victim']*100/actual_tag_count['LOC_Victim'])+'%')

======= Accuracy Class Wise =====================

Class    Matched Total %
--------------------------
PER_Others: 1136 1689 67%
PER_Victim: 40 251 15%
PER_Accused: 339 812 41%
ORG_Victim: 0 150 0%
ORG_Accused: 409 613 66%
ORG_Others: 1102 2122 51%
LOC_Accused: 0 178 0%
LOC_Others: 879 1724 50%
LOC_Event: 478 1338 35%
LOC_Victim: 0 27 0%
